In [2]:
import numpy as np
import pickle
import matplotlib
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, rc
from matplotlib.colors import LinearSegmentedColormap
from typing import cast
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from matplotlib import colormaps as cm
from matplotlib.ticker import LinearLocator
import os
from mpl_toolkits.mplot3d import Axes3D
from typing import cast

In [13]:
def initc(x, y):  # Initial condition

    u0 = np.ones((Mx, My))
    for i in range(Mx):
        for j in range(My):
            X, Y = x[i], y[j]
            u0[i,j] = np.sin(X+Y)+np.sin(X)+np.sin(Y)

    return u0

def wavenum(Mx, My):  # Wavenumber in Fourier space

    dk = np.pi/l
    kx = np.hstack((np.arange(0.,Mx/2.+1.),np.arange(-Mx/2.+1.,0.))).T*dk
    ky = np.hstack((np.arange(0.,My/2.+1.),np.arange(-My/2.+1.,0.))).T*dk

    return kx, ky 

def weights(x, y):  # Spatial integration weights 

    weights = np.zeros((Mx, My))
    nx = len(x)
    ny = len(y)
    dx = np.ones_like(x)
    dy = np.ones_like(y)

    for i in range(nx-1):
        dx[i] = x[i+1] - x[i]
        
    dx = np.delete(dx, [len(x)-1], None)
    
    for j in range(ny-1):
        dy[j] = y[j+1] - y[j]
        
    dy = np.delete(dy, [len(y)-1], None)
    
    for k in range(nx):
        for l in range(ny):
            if k == 0 and l == 0:
                weights[k,l] = dx[0]*dy[0]/4.
            elif k == 0 and l == ny-1:
                weights[k,l] = dx[0]*dy[-1]/4.
            elif k == nx-1 and l == 0:
                weights[k,l] = dx[-1]*dy[0]/4.
            elif k == nx-1 and l == ny-1:
                weights[k,l] = dx[-1]*dy[-1]/4.
            elif k == 0 and 0<l<ny-1: 
                weights[k,l] = dx[0]*(dy[l-1]+dy[l])/4.
            elif k == nx-1 and 0<l<ny-1:
                weights[k,l] = dx[-1]*(dy[l-1]+dy[l])/4.   
            elif 0<k<nx-1 and l == 0:
                weights[k,l] = (dx[k-1]+dx[k])*dy[0]/4.
            elif 0<k<nx-1 and l == ny-1:
                weights[k,l] = (dx[k-1]+dx[k])*dy[-1]/4.          
            else:
                weights[k,l] = (dx[k-1]+dx[k])*(dy[l-1]+dy[l])/4.

    return weights

def kssol(u0):  # Solves Kuramoto-Sivashinsky equation in 2-D Fourier space

    kx, ky = wavenum(Mx, My)
    nkx = len(kx)
    nky = len(ky)

    k2 = np.ones((Mx, My))
    kX = np.ones((Mx, My))
    kY = np.ones((Mx, My))
    for i in range(nkx):
        for j in range(nky):
            KX, KY = kx[i], ky[j] 
            k2[i,j] = -(KX**2)-((nu2/nu1)*(KY**2))+(nu1*((KX**4)+(2.*(nu2/nu1)*(KX**2)*(KY**2))+((nu2/nu1)**2*(KY**4))))    
            kX[i,j] = KX
            kY[i,j] = KY

    u0spec = np.fft.fft2(u0)  					# Initial condition in Fourier space

    nlinspecx = np.zeros((Mx, My, nt+1), dtype='complex')       # Nonlinear part x
    nlinspecy = np.zeros((Mx, My, nt+1), dtype='complex')       # Nonlinear part y
    NONLINEAR_PART = True
    MEAN = False

    if NONLINEAR_PART:
      nlinspecx[:,:,0] = -0.5*np.fft.fft2(np.absolute(np.fft.ifft2(1j*kX*u0spec))*np.absolute(np.fft.ifft2(1j*kX*u0spec))) 
      nlinspecy[:,:,0] = -0.5*(nu2/nu1)*np.fft.fft2(np.absolute(np.fft.ifft2(1j*kX*u0spec))*np.absolute(np.fft.ifft2(1j*kX*u0spec)))
    A = np.ones((Mx, My)) 
 
    u = np.zeros((Mx, My, nt+1), dtype='complex')		# Variable in Fourier space
    u[:,:,0] = u0spec
    ur = np.zeros((Mx, My, nt+1), dtype='complex')		# Variable in real space
    ur[:,:,0] = u0
    en = np.zeros((nt+1))					# Energy calculation
    wt = weights(x,y)
    ur2 = ur[:,:,0]*ur[:,:,0]
    en[0] = np.dot(wt.flatten(), ur2.flatten())
    nlin = np.zeros((nt+1))

    for i in range(nt):
        if i%1000 == 0:
          print(i)
        if i==0:
            u[:,:,i+1] = (u[:,:,i] + (dt*(nlinspecx[:,:,i] + nlinspecy[:,:,i] + (c*A*u[:,:,i]))))/(A + (dt*(k2+(c*A)))) 
            ur[:,:,i+1] = np.fft.ifft2(u[:,:,i+1]).real
            if MEAN:
              ur[:,:,i+1] = ur[:,:,i+1] - ((1./(4.*(np.pi**2)))*np.dot(wt.flatten(), ur[:,:,i+1].flatten())*A) 
            ur2 = ur[:,:,i+1]*ur[:,:,i+1]
            en[i+1] = np.dot(wt.flatten(), ur2.flatten())
        else:
            u[:,:,i] = np.fft.fft2(ur[:,:,i])
            if NONLINEAR_PART:
              nlinspecx[:,:,i] = -0.5*np.fft.fft2(np.absolute(np.fft.ifft2(1j*kX*u[:,:,i]))*np.absolute(np.fft.ifft2(1j*kX*u[:,:,i])))
              nlinspecy[:,:,i] = -0.5*(nu2/nu1)*np.fft.fft2(np.absolute(np.fft.ifft2(1j*kY*u[:,:,i]))*np.absolute(np.fft.ifft2(1j*kY*u[:,:,i])))
            u[:,:,i+1] = ((4.*u[:,:,i]) - u[:,:,i-1] + (4.*dt*(nlinspecx[:,:,i] + nlinspecy[:,:,i] + (c*A*u[:,:,i]))) - (2.*dt*(nlinspecx[:,:,i-1] + nlinspecy[:,:,i-1] + (c*A*u[:,:,i-1]))))/((3.*A) + (2.*dt*(k2+(c*np.ones_like(k2)))))
            ur[:,:,i+1] = np.fft.ifft2(u[:,:,i+1]).real
            if MEAN:
              ur[:,:,i+1] = ur[:,:,i+1] - ((1./(4.*(np.pi**2)))*np.dot(wt.flatten(), ur[:,:,i+1].flatten())*A)
            ur2 = ur[:,:,i+1]*ur[:,:,i+1]
            en[i+1] = np.dot(wt.flatten(), ur2.flatten())

    return ur, en

In [28]:
# Bifurcation parameters
nu1 = 1.             							# nu1 = (pi/Lx)^2
nu2 = 1.								# nu2 = (pi/Ly)^2

# Number of modes
Mx = 8                            					# Number of modes in x
My = 8                            					# Number of modes in y 

c = 100.                                            # Coefficient to ensure the positive definiteness of linear matrix A

# Run time 
Tf = 5.                          					# Final time
nt = 1000                         					# Number of time steps

Lx = np.pi/np.sqrt(nu1)                     				# Size of domain in x
Ly = np.pi/np.sqrt(nu2)	                     				# Size of domain in y

# Cell size
l = np.pi
dx = (2.*l)/(Mx)                    					# Grid spacing in x
dy = (2.*l)/(My) 							# Grid spacing in y

# Grid  
x = np.arange(0., Mx)*dx       						# Grid points in x
y = np.arange(0., My)*dy						# Grid points in y
X, Y = np.meshgrid(x, y, indexing='ij')					# Meshgrid in x-y
                 					
# Step-size
dt = Tf/nt                        					# Size of the time step 

t = np.linspace(0., Tf, nt+1)

In [15]:
k2 = np.ones((Mx, My))
np.ones_like(k2)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

In [16]:
kx, ky = wavenum(Mx, My)
nkx = len(kx)
nky = len(ky)

k2 = np.ones((Mx, My))
kX = np.ones((Mx, My))
kY = np.ones((Mx, My))
for i in range(nkx):
    for j in range(nky):
        KX, KY = kx[i], ky[j] 
        k2[i,j] = -(KX**2)-((nu2/nu1)*(KY**2))+(nu1*((KX**4)+(2.*(nu2/nu1)*(KX**2)*(KY**2))+((nu2/nu1)**2*(KY**4))))    
        kX[i,j] = KX
        kY[i,j] = KY

In [17]:
kY

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., -7., -6., -5., -4.,
        -3., -2., -1.],
       [ 0.,  1.,  2.,  3.,  4

In [18]:
u0 = initc(x,y)

ur, en = kssol(u0)

0


/tmp/ipykernel_33049/2270947198.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  en[0] = np.dot(wt.flatten(), ur2.flatten())
/tmp/ipykernel_33049/2270947198.py:107: ComplexWarning: Casting complex values to real discards the imaginary part
  en[i+1] = np.dot(wt.flatten(), ur2.flatten())
/tmp/ipykernel_33049/2270947198.py:118: ComplexWarning: Casting complex values to real discards the imaginary part
  en[i+1] = np.dot(wt.flatten(), ur2.flatten())


In [19]:
ur[:,:,-1]

array([[-3.05154924+0.j, -2.68307181+0.j, -2.32659146+0.j,
        -2.040273  +0.j, -1.87410108+0.j, -1.85840665+0.j,
        -1.99610426+0.j, -2.26191622+0.j, -2.60892785+0.j,
        -2.97965974+0.j, -3.31747907+0.j, -3.57514082+0.j,
        -3.71944519+0.j, -3.73288257+0.j, -3.61386239+0.j,
        -3.37670975+0.j],
       [-2.68305005+0.j, -2.31457366+0.j, -1.95809619+0.j,
        -1.67178205+0.j, -1.50561522+0.j, -1.48992579+0.j,
        -1.62762741+0.j, -1.89344174+0.j, -2.24045376+0.j,
        -2.61118411+0.j, -2.94900031+0.j, -3.20665786+0.j,
        -3.35095758+0.j, -3.36439046+0.j, -3.24536652+0.j,
        -3.00821139+0.j],
       [-2.32678135+0.j, -1.95830783+0.j, -1.60183454+0.j,
        -1.31552528+0.j, -1.14936322+0.j, -1.1336776 +0.j,
        -1.27138142+0.j, -1.53719598+0.j, -1.88420632+0.j,
        -2.25493342+0.j, -2.59274538+0.j, -2.85039836+0.j,
        -2.99469379+0.j, -3.00812323+0.j, -2.88909715+0.j,
        -2.65194147+0.j],
       [-2.04089083+0.j, -1.67242163+

In [20]:
weights(x,y)

array([[0.03855314, 0.07710628, 0.07710628, 0.07710628, 0.07710628,
        0.07710628, 0.07710628, 0.07710628, 0.07710628, 0.07710628,
        0.07710628, 0.07710628, 0.07710628, 0.07710628, 0.07710628,
        0.03855314],
       [0.07710628, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.07710628],
       [0.07710628, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.07710628],
       [0.07710628, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.07710628],
       [0.07710628, 0.15421257, 0.15421257, 0.15421257, 0.15421257,
        0.15421257, 0.15421257, 

In [29]:
nx = len(x)
ny = len(y)
dx = np.ones_like(x)
dy = np.ones_like(y)

for i in range(nx-1):
    dx[i] = x[i+1] - x[i]

dx = np.delete(dx, [len(x)-1], None)

In [30]:
dx

array([0.78539816, 0.78539816, 0.78539816, 0.78539816, 0.78539816,
       0.78539816, 0.78539816])